In [2]:
!pip install selenium
!pip install keyboard

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 89.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 384.9/384.9 KB 35.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 19.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 KB 8.5 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.1/58.1 KB 7.4 MB/s eta 0:00:00


In [3]:
!apt-get update
!apt-get install -y wget curl
!wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
!sudo apt-get install libu2f-udev
!sudo dpkg -i google-chrome-stable_current_amd64.deb
!google-chrome --version


Get:1 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Get:2 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease [18.1 kB]
Get:3 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  Release
Hit:7 http://archive.ubuntu.com/ubuntu focal InRelease
Get:8 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Get:12 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Get:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease [24.3 kB]
Get:14 http://security.ubuntu.co

In [5]:
!pip install paho-mqtt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.4/99.4 KB 11.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for paho-mqtt: filename=paho_mqtt-1.6.1-py3-none-any.whl size=62132 sha256=bfe2856291602c2e7b281a5cc0cfcca623e6b62a823eda2d54c4ff1b40290fac
  Stored in directory: /root/.cache/pip/wheels/6a/48/01/c895c027e9b9367ec5470fbf371ee56e795a49ac6a19aa4c9f
Successfully built paho-mqtt


In [ ]:
from selenium import webdriver
import time
import threading
import keyboard
import paho.mqtt.client as mqtt
import json

options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
options.add_argument("lang=en")
options.add_argument("start-maximized")
options.add_argument("disable-infobars")
options.add_argument("--disable-extensions")
options.add_argument("--incognito")
options.add_argument("--disable-blink-features=AutomationControlled")

driver = webdriver.Chrome('chromedriver',options=options)
driver.get("https://tinyurl.com/QU-research")

button1 = driver.find_element("id", "part1")
button2 = driver.find_element("id", "part2")
button3 = driver.find_element("id", "part3")
button4 = driver.find_element("id", "part4")
button5 = driver.find_element("id", "part5")
button6 = driver.find_element("id", "part6")

click_count = 0
running = False

def click_buttons(sleep_time):
    global click_count, running
    try:
        while running:
            button1.click()
            button2.click()
            button3.click()
            button4.click()
            button5.click()
            button6.click()
            click_count += 6
            print(click_count)
            time.sleep(sleep_time)
    except Exception as e:
        print(f"Number of clicks: {click_count}")

def on_connect(client, userdata, flags, rc):
    print("Connected with result code "+str(rc))
    client.subscribe("NPRP/US/QU/bot")

def on_message(client, userdata, msg):
    global running, click_count
    payload = msg.payload.decode()
    if msg.topic == "NPRP/US/QU/bot":
        if payload.startswith("start+") and not running:
            try:
                sleep_time = float(payload[6:])
                running = True
                thread = threading.Thread(target=click_buttons, args=(sleep_time,))
                thread.daemon = True
                thread.start()
            except ValueError:
                print("Invalid sleep time")
        elif payload == "stop" and running:
            running = False
            client.publish("NPRP/US/QU/bot", json.dumps({"click_count": click_count}))
            click_count = 0


client = mqtt.Client()
client.on_connect = on_connect
client.on_message = on_message
client.connect("broker.hivemq.com")

client.loop_forever()

# Close the browser window
driver.quit()


Connected with result code 0
6
12
18
24
30
36
42
48
54
60
66
72
78
84
90
96
102
108
114
120
126
6
12
18
24
30
